# Data types

In most cases, the data type (`dtype`) of a [Variable](../generated/classes/scipp.Variable.rst) is derived from the data.
For instance when passing a [numpy array](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) to scipp, scipp will use the `dtype` provided by numpy:

In [ ]:
import numpy as np
import scipp as sc

var = sc.Variable(dims=['x'], values=np.arange(4.0))
var.dtype

In [ ]:
var = sc.Variable(dims=['x'], values=np.arange(4))
var.dtype

The `dtype` may also be specified using a keyword argument to `sc.Variable` and most [creation functions](./creation-functions.rst#creation-functions).
It is possible to use scipp's own `scipp.dtype`, [numpy.dtype](https://numpy.org/doc/stable/reference/generated/numpy.dtype.html), or (where a numpy equivalent exists) a string:

In [ ]:
var = sc.zeros(dims=['x'], shape=[2], dtype=sc.dtype.float32)
var.dtype

In [ ]:
var = sc.zeros(dims=['x'], shape=[2], dtype=np.dtype(np.float32))
var.dtype

In [ ]:
var = sc.zeros(dims=['x'], shape=[2], dtype='float32')
var.dtype

Scipp supports common dtypes like

- `float32`, `float64`
- `int32`, `int64`
- `bool`
- `string`
- `datetime64`

It is also possible to nest Variables, DataArrays, or Datasets inside of Variables. This is useful for storing attributes in DataArrays and Datasets. But there is only limited interoperability with numpy in those cases.

In [ ]:
var = sc.scalar(sc.zeros(dims=['x'], shape=[2], dtype='float64'))
var

You can get a full list using 
```python
[s for s in dir(sc.dtype) if not s.startswith('__')]
```
but note that many of those dtypes are only meant for internal use.

## Dates and Times

Scipp has a special dtype for time-points, `sc.dtype.datetime64`.
Variables can be constructed from integers which encode the time since the [Unix epoch](https://en.wikipedia.org/wiki/Unix_time):

In [ ]:
sc.scalar(value=0, unit=sc.units.s, dtype=sc.dtype.datetime64)

In [ ]:
sc.scalar(value=681794055, unit=sc.units.s, dtype=sc.dtype.datetime64)

Datetime variables always need a temporal unit and that unit determines how the integer that is passed to `value=` is interpreted:

In [ ]:
var = sc.scalar(value=681794055, unit=sc.units.ns, dtype=sc.dtype.datetime64)
var

Datetime elements are automatically converted to and from [numpy.datetime64](https://numpy.org/doc/stable/reference/arrays.datetime.html#basic-datetimes) objects:

In [ ]:
var.value

In [ ]:
now = sc.scalar(value=np.datetime64('now'))
now

Note that `now` has unit `s` even though we did not specify it.
The unit was deduced from the `numpy.datetime64` object which encodes a unit of its own.

### Operations

Variables containing datetimes only support a limited set of operations as it makes no sense to, for instance, add two time points.
In contrast to numpy, scipp does not have a separate type for time differences.
Those are simply encoded by integer Variables with a temporal unit.

In [ ]:
a = sc.scalar(value=np.datetime64('2021-03-14', 'ms'))
b = sc.scalar(value=np.datetime64('2000-01-01', 'ms'))
a - b

In [ ]:
try:
    a + b
except sc.DTypeError as err:
    print(err)

In [ ]:
a + sc.scalar(value=123, unit='ms')

### Time zones

Scipp does not support manual handling of time zones.
All datetime objects are assumed to be in UTC.
Scipp does not look at your local time zone, thus the following will always produce 12:30 on 2021-03-09 UTC no matter where you are when you run this code:

In [ ]:
sc.scalar(value=np.datetime64('2021-09-03T12:30:00'))